In [23]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /Users/abpuccini/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages (0.0)


In [24]:
# Install joblib. This will be used to save a model. 
!pip install joblib

In [25]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [26]:
df = pd.read_csv("../Resources/exoplanet_data.csv")

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Display dataframe
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select features and label

In [27]:
# Set features as X value and label as y value
X = df.drop(columns=['koi_disposition'])
y = df['koi_disposition']

# Create a Train Test Split

Use `koi_disposition` for the y values

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [29]:
from sklearn.preprocessing import MinMaxScaler

# Scale your data
X_minmax = MinMaxScaler().fit(X_train)

X_train_scaled = X_minmax.transform(X_train)
X_test_scaled = X_minmax.transform(X_test)

# Create Model

In [30]:
# Support vector machine linear classifier
from sklearn.svm import SVC

# Create a Logistic Regression Model
model = SVC(kernel='linear')

# Fit (train) or model using the training data
model.fit(X_train_scaled, y_train)

SVC(kernel='linear')

In [31]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.8439824527942018
Testing Data Score: 0.8415331807780321


# Identify insignificant variables

In [32]:
# from sklearn.svm import SVC
from sklearn.feature_selection import RFE

# create the RFE model
rfe = RFE(model)
rfe = rfe.fit(X_train_scaled, y_train)
# summarize the selection of the attributes
rfe_df = pd.DataFrame({
    'features': X.columns,
    'support': rfe.support_,
    'ranking': rfe.ranking_
})
rfe_df

,features,support,ranking
0,koi_fpflag_nt,True,1
1,koi_fpflag_ss,True,1
2,koi_fpflag_co,True,1
3,koi_fpflag_ec,True,1
4,koi_period,True,1
5,koi_period_err1,True,1
6,koi_period_err2,False,2
7,koi_time0bk,False,4
8,koi_time0bk_err1,True,1
9,koi_time0bk_err2,True,1


In [33]:
# Identify insignificant features
features = rfe_df.loc[rfe_df['support'] == False]
drop_columns = list(features['features'])

# Train the Model

#### Using only significant features

In [34]:
drop_columns.append('koi_disposition')
drop_columns

['koi_period_err2',
 'koi_time0bk',
 'koi_impact',
 'koi_impact_err2',
 'koi_depth',
 'koi_depth_err1',
 'koi_depth_err2',
 'koi_prad',
 'koi_prad_err1',
 'koi_prad_err2',
 'koi_insol',
 'koi_insol_err1',
 'koi_insol_err2',
 'koi_tce_plnt_num',
 'koi_slogg_err1',
 'koi_srad',
 'koi_srad_err2',
 'ra',
 'dec',
 'koi_kepmag',
 'koi_disposition']

In [35]:
# Set features as X value and label as y value
X = df.drop(columns=drop_columns)
y = df['koi_disposition']

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [37]:
from sklearn.preprocessing import MinMaxScaler

# Scale your data
X_minmax = MinMaxScaler().fit(X_train)

X_train_scaled = X_minmax.transform(X_train)
X_test_scaled = X_minmax.transform(X_test)

In [38]:
# Create a Logistic Regression Model
model = SVC(kernel='linear')

# Fit (train) or model using the training data
model.fit(X_train_scaled, y_train)

SVC(kernel='linear')

In [39]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.8436009917985886
Testing Data Score: 0.8426773455377574


In [50]:
# print prediction results
from sklearn.metrics import classification_report

predictions = model.predict(X_test_scaled)
print(classification_report(y_test, predictions)) 

                precision    recall  f1-score   support

     CANDIDATE       0.75      0.57      0.65       422
     CONFIRMED       0.67      0.79      0.73       450
FALSE POSITIVE       0.98      1.00      0.99       876

      accuracy                           0.84      1748
     macro avg       0.80      0.79      0.79      1748
  weighted avg       0.84      0.84      0.84      1748



# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [51]:
from sklearn.model_selection import GridSearchCV
from sklearn.exceptions import FitFailedWarning
import warnings
warnings.simplefilter('always', FitFailedWarning)

# Create the GridSearchCV model: https://www.geeksforgeeks.org/svm-hyperparameter-tuning-using-gridsearchcv-ml/

# Defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['linear']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 

In [52]:
# Source: https://scikit-learn.org/0.22/modules/generated/sklearn.exceptions.FitFailedWarning.html
# Fit the model using the grid search estimator.
with warnings.catch_warnings(record=True) as w:
    try:
        grid.fit(X_train_scaled, y_train)  # This will raise a ValueError since C is < 0
    except ValueError:
         pass
    print(repr(w[-1].message))

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ..................C=0.1, gamma=1, kernel=linear; total time=   0.3s
[CV 2/5] END ..................C=0.1, gamma=1, kernel=linear; total time=   0.2s
[CV 3/5] END ..................C=0.1, gamma=1, kernel=linear; total time=   0.2s
[CV 4/5] END ..................C=0.1, gamma=1, kernel=linear; total time=   0.2s
[CV 5/5] END ..................C=0.1, gamma=1, kernel=linear; total time=   0.3s
[CV 1/5] END ................C=0.1, gamma=0.1, kernel=linear; total time=   0.3s
[CV 2/5] END ................C=0.1, gamma=0.1, kernel=linear; total time=   0.3s
[CV 3/5] END ................C=0.1, gamma=0.1, kernel=linear; total time=   0.2s
[CV 4/5] END ................C=0.1, gamma=0.1, kernel=linear; total time=   0.2s
[CV 5/5] END ................C=0.1, gamma=0.1, kernel=linear; total time=   0.2s
[CV 1/5] END ...............C=0.1, gamma=0.01, kernel=linear; total time=   0.2s
[CV 2/5] END ...............C=0.1, gamma=0.01, 

[CV 2/5] END .................C=1000, gamma=1, kernel=linear; total time=   1.7s
[CV 3/5] END .................C=1000, gamma=1, kernel=linear; total time=   2.1s
[CV 4/5] END .................C=1000, gamma=1, kernel=linear; total time=   1.2s
[CV 5/5] END .................C=1000, gamma=1, kernel=linear; total time=   1.4s
[CV 1/5] END ...............C=1000, gamma=0.1, kernel=linear; total time=   1.5s
[CV 2/5] END ...............C=1000, gamma=0.1, kernel=linear; total time=   1.5s
[CV 3/5] END ...............C=1000, gamma=0.1, kernel=linear; total time=   1.7s
[CV 4/5] END ...............C=1000, gamma=0.1, kernel=linear; total time=   1.1s
[CV 5/5] END ...............C=1000, gamma=0.1, kernel=linear; total time=   1.3s
[CV 1/5] END ..............C=1000, gamma=0.01, kernel=linear; total time=   1.3s
[CV 2/5] END ..............C=1000, gamma=0.01, kernel=linear; total time=   1.5s
[CV 3/5] END ..............C=1000, gamma=0.01, kernel=linear; total time=   1.7s
[CV 4/5] END ..............C

IndexError: list index out of range

In [53]:
print(f'Best Parameter: {grid.best_params_}')
print(f'Best Model Score: {grid.best_score_}')

Best Parameter: {'C': 1000, 'gamma': 1, 'kernel': 'linear'}
Best Model Score: 0.8832746927280798


In [56]:
predictions = grid.predict(X_test_scaled)
print(classification_report(y_test, predictions)) 

                precision    recall  f1-score   support

     CANDIDATE       0.82      0.69      0.75       422
     CONFIRMED       0.75      0.83      0.79       450
FALSE POSITIVE       0.98      1.00      0.99       876

      accuracy                           0.88      1748
     macro avg       0.85      0.84      0.84      1748
  weighted avg       0.88      0.88      0.88      1748



# Save the Model

In [54]:
import joblib
from joblib import dump, load

# save model
filename = '../Models/svm.sav'
joblib.dump(grid, filename)

['../Models/svm.sav']